In [182]:
import pandas as pd
import json
import requests
from tqdm import tqdm
from datetime import datetime
df = pd.read_excel('20221011_upload_pessoa_not_mapped.xlsx',sheet_name="Import", converters={'PHONE_IBD':str})
# Convert Phone number to valid value 
df['PHONE_IBD'] = df['PHONE_IBD'].str.replace('\+55','55-',n=2,regex=True)
#print(df['PHONE_IBD'])
!python -V

Python 3.9.12


In [183]:
#df['externalId'] = ""  # if no id provided  API will generate
df['marketingClientId']=None  # for confirmation
df['parentClientId']=None     # for confirmation
df['password']=None           # for confirmation
df['industry']="Other"        # for confirmation
df['accountManager']=""       # for confirmation
df['accountStatus']="Active"
df['businessUnit']="IBD"  # for confirmation
df['leadType']="New"      # for confirmation
df['inspectionPotentialTO']="200"  # for confirmation
df['auditPotentialTO']="100"    # for confirmation
df['labTestPotentialTO']="400"  # for confirmation
df['saberPotentialTO']="300"    # for confirmation
df['comments']=None    # for confirmation
df['altEmail']=None    # for confirmation
df['accountingEmail']=None
df['salutation']=None    # for confirmation
df['jobTitle']="Manager"      # for confirmation (Mandatory)  use Manager by default
df['keyAccount']=False   # for confirmation
df['unitNumber']=None    # for confirmation
df['municipality']=None  # for confirmation
df['districtCode']=None  # for confirmation
df['streetType']=None    # for confirmation
df['acquisition']=None   # for confirmation

#streetName = STREET_IBD  available value is number


renamed_df = df.rename(columns = {"EXTERNAL_COMPANY_ID_IBD":"externalId"
                                  ,"FIRSTNAME_IBD":"firstName"
                                  ,"LASTNAME_IBD":"lastName"
                                  ,"EMAIL":"email"
                                  ,"COMPANYNAME_IBD":"companyName"
                                  ,"PHONE_IBD":"phoneNumber"
                                  ,"COUNTRY_IBD":"country"
                                  ,"ADDRESS_IBD":"address"
                                  ,"ZIP_IBD":"postCode"
                                  ,"LOGIN":"login"
                                  ,"STATE_CODE":"state"
                                  ,"CITY_IBD":"city"
                                  ,"CITY_CODE":"cityCode"
                                  ,"STATE_NAME":"stateInLocal"
                                  ,"COMPANY_SHORT_NAME_IBD":"companyNameInLocal"
                                  ,"HOUSE_NUMBER_IBD":"houseNumber"
                                  ,"DISTRICT_IBD":"district"
                                  ,"STREET_IBD":"streetName"
                                  ,"TAX_SITUATION_IBD":"TAX_SITUATION"
                                  ,"TAX_NUMBER_IBD":"VAT_NUMBER"
                                  ,"STATE_TAX_ID_IBD":"FEDERAL_TAX_ID"})

arranged_renamed_df = renamed_df[['externalId'
                                  ,'companyName'
                                  ,'marketingClientId'
                                  ,'parentClientId'
                                  ,'login'
                                  ,'password'
                                  ,'industry'
                                  ,'accountManager'
                                  ,'accountStatus'
                                  ,'businessUnit'
                                  ,'leadType'
                                  ,'inspectionPotentialTO'
                                  ,'auditPotentialTO'
                                  ,'labTestPotentialTO'
                                  ,'saberPotentialTO'
                                  ,'comments'
                                  ,'country'
                                  ,'state'
                                  ,'city'
                                  ,'address'
                                  ,'email'
                                  ,'altEmail'
                                  ,'accountingEmail'
                                  ,'phoneNumber'
                                  ,'postCode'
                                  ,'salutation'
                                  ,'firstName'
                                  ,'lastName'
                                  ,'jobTitle'
                                  ,'keyAccount'
                                  ,'unitNumber'
                                  ,'municipality'
                                  ,'cityCode'
                                  ,'districtCode'
                                  ,'streetType'
                                  ,'acquisition'
                                  ,'district'
                                  ,'streetName'
                                  ,'houseNumber'
                                  ,'TAX_SITUATION'
                                  ,'VAT_NUMBER'
                                  ,'FEDERAL_TAX_ID']]
arranged_renamed_df.head(2)




,externalId,companyName,marketingClientId,parentClientId,login,password,industry,accountManager,accountStatus,businessUnit,...,cityCode,districtCode,streetType,acquisition,district,streetName,houseNumber,TAX_SITUATION,VAT_NUMBER,FEDERAL_TAX_ID
0,26,Fazenda Amway Nutrilite do Brasil Ltda,None,None,ibd_fazenda_amway_nutrilite_do_brasil_ltda,None,Other,,Active,IBD,...,1461.0,None,None,None,Zona Rural,26,S/N,CNPJ,02.038.405.0001-44,06.668.572-8
1,89,Companhia Cacique de Café Solúvel,None,None,ibd_companhia_cacique_de_cafe_soluvel,None,Other,,Active,IBD,...,3149.0,None,None,None,Pq Indl Cacique,26,100,CNPJ,78.588.415/0001-15,60.102.504-37


In [184]:
## Script to check already exixt record based on login, and company_name ( external_id)
#SELECT  cc.id company_id, cc.company_name, ccu.user_id, gu.first_name||' '||gu.last_name company_user, gu.personal_email,  gu.login company_login, cem.id as main_external_id, cem.external_system, cem.external_id, cem.qima_client_id
#FROM AIDATAUSER.CRM_COMPANY_USER ccu  
#   inner join AIDATAUSER.CRM_COMPANY cc on ccu.company_id = cc.id
#   left join AIDATAUSER.GENERAL_USER gu on ccu.user_id = gu.id
#   left join AIDATAUSER.CLIENT_EXTERNAL_MAPPING cem on cc.id = cem.qima_client_id
# -- where cc.id in ('669BD08995754C8B82AF20ED4CEE12FB')   
# -- where ccu.user_id in ('FFE1949B608846A6B361F7A848216137')
#  where gu.login = 'ibd_antonio_carlos_da_costa_sitio_sao_sebastiao'
# -- where cc.company_name like '%Tanac%'

# TUIUIU EMPREENDIMENTOS AGROPECUARIOS LTDA external_id is null  and in excel Tuiuiu Empreendimentos Agropecuários LTDA external_id 1986
# Cooperativa dos Produtores e Colhedores Orgânicos de Parnaíba -> already exist but misspelled  'Colheres' 
# PROD ROBERTO NASCIMENTO OLIVEIRA   5294  and excel Roberto Nascimento Oliveira    5047
#Masi Agricola SPA ---> external_id 5910 existing on db (PP and PROD) and 5873 on excel
#PP and PROD Iwao Akamatsu e outros  3603  and in excel Iwao Akamatsu e Outros (Yuri Cogumelos)  5936
#PP and PROD Cooperativa Agricola de Formosa do Rio Preto - Coafor external_id 9332 and in excel - Cooperativa Agrícola De Formosa do Rio Preto - COAFOR       6228
#PP and PROD COOPALM - Coop. Dos Prod. de Palmito do Baixo Sul da Bahia - COAFOR    internal_id   2929 and in Excel - Cooperativa dos Produtores  de Palmito do Baixo Sul da Bahia ( COOPALM)    6703
#PP and Prod Tanac S/A and in  excel Tanac S.A
#PP and Staging Associação dos Produtores de Cafés Especiais do Sul de Minas-APROCEM  and in excel Associação dos Produtores de Cafés Especiais do Sul de Minas (Aprocem)
#PP and staging LOUISA LEE external_id 7654  and in excel Louisa Lee (Recanto Ecológico)  external_id 7658
#COMPANHIA TEXTIL DE CASTANHAL externale_id 7763 and in excel Companhia Têxtil de Castanhal external_id 7764
#OSVALDO VIU SERRANO JUNIOR external_id 734 and in excel Osvaldo Viu Serrano Junior (Sítio Fugidos) external_id 7915
#ANTONIO CARLOS DA COSTA E OUTRA external_id 3644 and in excel Antonio Carlos da Costa (Sítio São Sebastião) external_id 7916
#Perboni S/A external_id 8772 and in excel Perboni S/A external_id 8802
#RENIERIS P.C external_id 9155 and in excel Renieris P.C external_id 8939
# USINA SAO FRANCISCO S/A  external_id 169 and in excel Usina São Francisco S/A (Native) external_id 9111
# IFB - Instituto de Fosfatados Biológicos Ltda.  external_id 1143 and in excel Instituto de Fosfatos Biológicos Ltda external_id 9113
#Chemyunion Química LTDA and in excel Chemyunion Ltda_1 external_id 9181
# Mandiervas Indústria e Comércio de Ervas Medicinais Ltda external_id 3999 and in excel Mandiervas Comercio De Ervas e Condimentos Ltda external_id 9547
# ERALDO DIAS DE CASTRO JUNIOR external_id 9041 and in excel Eraldo Dias de Castro Junior external_id 9613

companty_to_exclude= ['Tuiuiu Empreendimentos Agropecuários LTDA'
                      ,'Cooperativa dos Produtores e Colhedores Orgânicos de Parnaíba'
                      ,'Roberto Nascimento Oliveira'
                      ,'Masi Agricola SPA'
                      ,'Iwao Akamatsu e Outros (Yuri Cogumelos)'
                      ,'Cooperativa Agrícola De Formosa do Rio Preto - COAFOR'
                      ,'Cooperativa dos Produtores  de Palmito do Baixo Sul da Bahia ( COOPALM)'
                      ,'Tanac S.A'
                      ,'Associação dos Produtores de Cafés Especiais do Sul de Minas (Aprocem)'
                      ,'Louisa Lee (Recanto Ecológico)'
                      ,'Companhia Têxtil de Castanhal'
                      ,'Osvaldo Viu Serrano Junior (Sítio Fugidos)'
                      ,'Antonio Carlos da Costa (Sítio São Sebastião)'
                      ,'Perboni S/A'
                      ,'Renieris P.C'
                      ,'Usina São Francisco S/A (Native)'
                      ,'Instituto de Fosfatos Biológicos Ltda'
                      ,'Mandiervas Comercio De Ervas e Condimentos Ltda'
                      ,'Eraldo Dias de Castro Junior']
arranged_renamed_df = arranged_renamed_df[~arranged_renamed_df.companyName.isin(companty_to_exclude)]

In [185]:
#sample_record = arranged_renamed_df.head(1)
#filter_value = ['Fazenda Amway Nutrilite do Brasil Ltda',
#'Companhia Cacique de Café Solúvel',
#'Fernando Borges de Assis',
#'NUTRITION & SANTÉ DO BRASIL LTDA',
#'Coop. dos Produtores Org. do Sul da Bahia - CABRUCA',
#'Cláudio Carneiro Pinto',
#'Marfrig Global Foods S/A']
#filter_value = ['Nuncio Regino']
#filter_value = ['Ibis Agraria Ltda']
#filter_value = ['Cumberland Packing Corp.']
#filter_value = ['Ceres Enterprises Ltd.']
#record = arranged_renamed_df[arranged_renamed_df['companyName'].isin(filter_value)]
record = arranged_renamed_df
record 

,externalId,companyName,marketingClientId,parentClientId,login,password,industry,accountManager,accountStatus,businessUnit,...,cityCode,districtCode,streetType,acquisition,district,streetName,houseNumber,TAX_SITUATION,VAT_NUMBER,FEDERAL_TAX_ID
0,26,Fazenda Amway Nutrilite do Brasil Ltda,None,None,ibd_fazenda_amway_nutrilite_do_brasil_ltda,None,Other,,Active,IBD,...,1461.0,None,None,None,Zona Rural,26,S/N,CNPJ,02.038.405.0001-44,06.668.572-8
1,89,Companhia Cacique de Café Solúvel,None,None,ibd_companhia_cacique_de_cafe_soluvel,None,Other,,Active,IBD,...,3149.0,None,None,None,Pq Indl Cacique,26,100,CNPJ,78.588.415/0001-15,60.102.504-37
2,113,Fernando Borges de Assis,None,None,ibd_fernando_borges_de_assis,None,Other,,Active,IBD,...,4256.0,None,None,None,Várzea dos Cedros,25,s/n,CPF,01.499.018/0001-42,500.584.444-9
3,262,NUTRITION & SANTÉ DO BRASIL LTDA,None,None,ibd_nutrition_&_sante_do_brasil_ltda,None,Other,,Active,IBD,...,3013.0,None,None,None,Campo Fundo,2,896,CNPJ,81.727.414/0008-95,9068354010
4,470,Coop. dos Produtores Org. do Sul da Bahia - CA...,None,None,ibd_coop_dos_produtores_org_do_sul_da_bahia_ca...,None,Other,,Active,IBD,...,3618.0,None,None,None,Nossa Senhora da Vitória,2,2038,CNPJ,04.293.272/0001-88,055.657.534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,9822,Ibis Agraria Ltda,None,None,ibd_ibis_agraria_ltda_3,None,Other,,Active,IBD,...,345.0,None,None,None,Zona Rural,25,S/N,CNPJ,13.504.053/0008-68,28.735.761-9
263,9823,Ibis Agraria Ltda,None,None,ibd_ibis_agraria_ltda_4,None,Other,,Active,IBD,...,330.0,None,None,None,ZONA RURAL,25,S/N,CNPJ,13.504.053/0008-68,28.710.366-8
264,9827,Eliane Aparecida Da Luz 79147127953,None,None,ibd_eliane_aparecida_da_luz_79147127953,None,Other,,Active,IBD,...,2799.0,None,None,None,Rio Bonito (Pirabeiraba),9,1301,CNPJ,46.162.220/0001-12,NaN
265,9828,Inpasa Del Paraguay S.A.,None,None,ibd_inpasa_del_paraguay_sa,None,Other,,Active,IBD,...,NaN,None,None,None,-,25,S/N,CNPJ,80034565-7,137201761


In [186]:
param_generateLogin = "false"
param_loginStrategy = "email"
param_generatePassword = "true"
param_passwordStrategy = "random"
param_keepOriginalPassword = "false"
param_externalSystemName = "IBD"
param_businessUnitKey = "IBD"
param_externalIdByMd5 = "false"
param_allowUpdate = "true"

In [187]:
# Assign URL
post_url_new_account = f"'http://ppservice.qima.com/customer-service/v1.0/customer?generateLogin={param_generateLogin}&loginStrategy={param_loginStrategy}&generatePassword={param_generatePassword}&passwordStrategy={param_passwordStrategy}&keepOriginalPassword={param_keepOriginalPassword}&externalSystemName={param_externalSystemName}&businessUnitKey={param_businessUnitKey}&externalIdByMd5={param_externalIdByMd5}&allowUpdate={param_allowUpdate}'"
print(post_url_new_account)

post_url_sync_netsuite="'https://ppservice.qima.com/customer-service/customer/update-invoicing-fields'"
print(post_url_sync_netsuite)


'http://ppservice.qima.com/customer-service/v1.0/customer?generateLogin=false&loginStrategy=email&generatePassword=true&passwordStrategy=random&keepOriginalPassword=false&externalSystemName=IBD&businessUnitKey=IBD&externalIdByMd5=false&allowUpdate=true'
'https://ppservice.qima.com/customer-service/customer/update-invoicing-fields'


In [188]:
v_externalId = []
v_companyName = []
v_login = []
v_city = []
v_phoneNumber = []
v_clientId_append = []
v_clientId = []

for i in tqdm(record.index):
    data = json.dumps(json.loads(record.loc[i].to_json(orient='index')), indent=4)   
    data = json.loads(data)
    post_new_account =eval(post_url_new_account)
    response = requests.post(post_new_account,json=data)
    #print(data)
    #print ('Status_code = ',response.status_code,'response content = ',response.content)   
    if response.status_code == 200:  ## 200 OK	Success on GET, PUT, or POST. Returned for a successful response
        get_customerId = json.loads(response.content)
        customerId = get_customerId['content']['clientId']        
        v_clientId_append.append(customerId)
        if(customerId and not customerId.isspace()):
            #print("The string is not empty",customerId,' Login ', [record.loc[i]['login']], ' Company Name ', [record.loc[i]['companyName']])
            if data['VAT_NUMBER'] is None:                  
                if data['FEDERAL_TAX_ID'] is None:
                     data_sync_netsuite = [{
                          "customerId":  customerId
                          ,"TAX_SITUATION": data['TAX_SITUATION']                          
                          }]
                else:
                    data_sync_netsuite = [{
                          "customerId":  customerId
                          ,"TAX_SITUATION": data['TAX_SITUATION']
                          ,"FEDERAL_TAX_ID":data['FEDERAL_TAX_ID']  
                          }]
            else:
                if data['FEDERAL_TAX_ID'] is None:
                    data_sync_netsuite = [{
                          "customerId":  customerId
                          ,"TAX_SITUATION": data['TAX_SITUATION']
                          ,"VAT_NUMBER": data['VAT_NUMBER']
                          }]
                else: 
                    data_sync_netsuite = [{
                          "customerId":  customerId
                          ,"TAX_SITUATION": data['TAX_SITUATION']   ## Mandatory field in QSP
                          ,"VAT_NUMBER": data['VAT_NUMBER']
                          ,"FEDERAL_TAX_ID":data['FEDERAL_TAX_ID']
                          }] 
            post_sync_netsuite =eval(post_url_sync_netsuite)
            response_sync_netsuite = requests.post(post_sync_netsuite,json=data_sync_netsuite)
            if response_sync_netsuite.status_code == 200:
                print()
                v_clientId = v_clientId_append
                v_externalId += [record.loc[i]['externalId']]
                v_companyName += [record.loc[i]['companyName']]
                v_login += record.loc[i]['login'],
                v_city += [record.loc[i]['city']]
                v_phoneNumber += [record.loc[i]['phoneNumber']]
                print ('Status_code = ',response_sync_netsuite.status_code,'response content = ',response_sync_netsuite.content)
    if response.status_code == 400:
        print ( 'Status_code Customer API= ',response.status_code,'response content = ',response.content, ' CompanyName ',record.loc[i]['companyName']
                ,' Status_code Netsuite = ',response_sync_netsuite.status_code, 'response content = ',response.content)
        break
                
df_create = pd.DataFrame({'clientId':v_clientId, 
    'externalId':v_externalId, 
    'companyName':v_companyName, 
    'login':v_login,
    'city':v_city, 
    'phoneNumber':v_phoneNumber})

today = datetime.now()
writer = pd.ExcelWriter(f'output/recorded_clients{today.year}_{today.month}_{today.day}_{today.hour}_{today.minute}_{today.second}.xlsx')
df_create.to_excel(writer, sheet_name='Sheet1')  
writer.save()

  0%|          | 1/248 [00:01<08:04,  1.96s/it]


Status_code =  200 response content =  b'{"customerId[F4FE0FEB6F4F457A8133B7195D099A67]":"invoicing fields updated successfully"}'


  1%|          | 2/248 [00:03<07:08,  1.74s/it]


Status_code =  200 response content =  b'{"customerId[097572F7E5EE4A0485375770420F92B2]":"invoicing fields updated successfully"}'


  1%|          | 3/248 [00:05<06:35,  1.62s/it]

...


Status_code =  200 response content =  b'{"customerId[AF44A55F98E84917AA09EE2DBD2E7271]":"invoicing fields updated successfully"}'


100%|█████████▉| 247/248 [03:50<00:00,  1.08it/s]


Status_code =  200 response content =  b'{"customerId[54A67E10D8F945CA80B7EC7C7177EB1A]":"invoicing fields updated successfully"}'


100%|██████████| 248/248 [03:51<00:00,  1.07it/s]


Status_code =  200 response content =  b'{"customerId[D3E78FA9809A482A9AC05B20E3460616]":"invoicing fields updated successfully"}'
